In [10]:
import networkx as nx
import pickle

file_path = 'C:/my_code/moduroad/moduroad_app/cashe_data/api_graph.pickle'

# Load the stored graph object
with open(file_path, 'rb') as file:
    api_graph = pickle.load(file)

# Assuming that api_graph is a NetworkX graph, let's iterate over all edges.
# We aim to change the 'weight' attribute to 'length' for each edge.
for u, v, data in api_graph.edges(data=True):
    # Store the 'weight' value
    weight = data['weight']
    # Assign the 'weight' value to a new attribute called 'length'
    data['length'] = weight
    # Now remove the 'weight' attribute
    del data['weight']


In [51]:
import pickle
import networkx as nx
file_path = 'C:/my_code/moduroad/moduroad_app/cashe_data/api_graph.pickle'

# Load the stored graph object
with open(file_path, 'rb') as file:
    api_graph = pickle.load(file)

for u, v, data in api_graph.edges(data=True):
    # Assign the 'weight' value to a new attribute called 'length'
    # Add 'slope' and 'stair' attributes, with default values of 0
    data['slope'] = 0
    data['stair_steep'] = 0
    data['sidewalk_curb'] = 0
    data['crosswalk_curb'] = 0
    data['bollard'] = 0
    data['access_zone'] = 0
    data['weight'] = 0
    # Now remove the 'weight' attribute
    

# api_graph가 이미 정의되어 있고 수정되었다고 가정

# 저장할 파일 경로 지정
file_path_to_save = 'C:/my_code/moduroad/moduroad_app/cashe_data/api_graph.pickle'

# api_graph 객체를 피클 파일로 저장
with open(file_path_to_save, 'wb') as file:
    pickle.dump(api_graph, file)


In [12]:
#엣지 데이터로 데이터프레임 으로 변환
import pandas as pd
import networkx as nx
import pickle
G=api_graph
# Graph 객체에서 엣지 정보를 불러와 DataFrame으로 변환
# 엣지 데이터는 (노드1, 노드2) 튜플의 리스트 형태이다
edges = list(G.edges(data=True))
# 데이터가 딕셔너리 형태의 속성(data=True)을 포함하는 경우
# 노드1, 노드2, 그리고 (키, 값) 쌍을 포함하는 딕셔너리를 포함한 튜플 형태인 경우 처리

# 엣지 정보를 DataFrame으로 변환 (노드1, 노드2, 속성 딕셔너리)
df_edges = pd.DataFrame(edges, columns=['Node1', 'Node2', 'Attributes'])

# 필요한 경우 속성 딕셔너리를 별도의 열로 확장할 수 있습니다
# 예를 들어, edge에 'weight' 속성이 있다면 다음과 같이 처리할 수 있습니다
# df_edges['Weight'] = df_edges['Attributes'].apply(lambda x: x.get('weight', None))

df_edges['Attributes'][0]


{'link_id': '28200000067',
 'geometry': <LINESTRING (126.369 37.463, 126.369 37.463)>,
 'length': 14.02555745870837}

In [15]:
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from shapely.strtree import STRtree
import pickle
import osmnx as ox
import folium
file_path = 'C:/my_code/moduroad/moduroad_app/cashe_data/api_graph.pickle'

with open(file_path, 'rb') as file:
    G = pickle.load(file)
G.graph['crs'] = 'epsg:4326'
# 장
obstacle_point = Point(	37.377069, 126.634596)


# 장애물 가까이에 있는 엣지 가중치 증가 함수 정의
def update_edge_weights(G, point, increment=1):
    # 가장 가까운 엣지 찾기
    nearest_edge_nodes = ox.distance.nearest_edges(G, X=point.y, Y=point.x)  # OSMnx는 (y, x) 형식으로 좌표를 사용합니다.
    print(nearest_edge_nodes)
    # nearest_edge_nodes는 (u, v, key) 형태의 튜플입니다.
    u, v, key = nearest_edge_nodes

    # 해당 엣지의 데이터 가져오기
    nearest_edge = G[u][v][key]
    print(nearest_edge)
    # 가장 가까운 엣지의 가중치 업데이트
    if nearest_edge:
        nearest_edge['bollard'] += increment
        print("change")
        print(nearest_edge)
        return (u, v, key), nearest_edge
    
m = folium.Map(location=[37.37463, 126.64481], zoom_start=14)    


modified_edge, edge_data = update_edge_weights(G, obstacle_point)
combined_line_string = edge_data['geometry']


# 합쳐진 LineString을 Folium 지도에 추가 (비어 있지 않은 경우에만 추가)
if combined_line_string:
    folium.GeoJson(combined_line_string, style_function=lambda x: {'color': 'blue', 'weight': 5}).add_to(m)
else:
    print("지도에 추가할 LineString이 없습니다.")
# 시작 지점에 마커 추가
folium.Marker(
    location=[obstacle_point.x,obstacle_point.y],
    icon=folium.Icon(color='red')
).add_to(m)

m

('28000761546', '28000762852', 0)
{'link_id': '28200362607', 'weight': 63.702618998460366, 'geometry': <LINESTRING (126.635 37.377, 126.635 37.377, 126.635 37.377, 126.635 37.377,...>}
change
{'link_id': '28200362607', 'weight': 68.70261899846037, 'geometry': <LINESTRING (126.635 37.377, 126.635 37.377, 126.635 37.377, 126.635 37.377,...>}


In [20]:
import pandas as pd# Load the DataFrame from the pickle file
pickle_file_path = 'C:/my_code/moduroad/data/obstacle/slope.pkl'  # Replace with your pickle file path
obstacle_df = pd.read_pickle(pickle_file_path)

# 모든 행의 데이터를 오른쪽으로 1칸씩 이동
obstacle_df = obstacle_df.shift(periods=1, axis='columns')

# 첫 번째 열 삭제 ('hg' 열이 이동 후 첫 번째 열임)
obstacle_df.drop(obstacle_df.columns[0], axis=1, inplace=True)

# 새로운 열 이름 지정 (예시에서 주어진 열 이름 사용)
new_columns = ['ctprvnnm', 'signgunm', 'signgucode', 'rdnmadr', 'lnmadr', 'latitude', 'longitude', 'institutionnm', 'phonenumeric', 'referencedate', 'instt_code', 'instt_nm', 'handicapcvntlyn', 'handicapcvntltype']
obstacle_df.columns = new_columns


obstacle_df

,ctprvnnm,signgunm,signgucode,rdnmadr,lnmadr,latitude,longitude,institutionnm,phonenumeric,referencedate,instt_code,instt_nm,handicapcvntlyn,handicapcvntltype
0,인천광역시,남동구,28200,,인천광역시 남동구 남촌도림동 648-5,37.421526,126.72953,인천광역시청,,2021-12-03,,,Y,경사로
1,인천광역시,남동구,28200,,인천광역시 남동구 논현2동 685-0,37.405426,126.723187,인천광역시청,,2021-12-03,,,Y,경사로
2,인천광역시,남동구,28200,,인천광역시 남동구 논현2동 688-0,37.405726,126.723106,인천광역시청,,2021-12-03,,,Y,경사로
3,인천광역시,미추홀구,28177,,인천광역시 미추홀구 관교동 520-0,37.445083,126.689948,인천광역시청,,2021-12-03,,,Y,경사로
4,인천광역시,미추홀구,28177,,인천광역시 미추홀구 관교동 520-0,37.445562,126.690162,인천광역시청,,2021-12-03,,,Y,경사로
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7751,인천광역시,미추홀구,28177,"인천광역시 미추홀구 낙섬중로 20, (용현동, 한양아파트)",인천광역시 미추홀구 용현5동 627-78,37.450717,126.638515,인천광역시청,,2021-12-03,,,Y,경사로
7752,인천광역시,남동구,28200,"인천광역시 남동구 담방로 29, (만수동, 광명아파트상가)",인천광역시 남동구 만수6동 1042-0,37.441782,126.736736,인천광역시청,,2021-12-03,,,Y,경사로
7753,인천광역시,남동구,28200,"인천광역시 남동구 장승남로 70-11, (만수동, 현대아파트상가)",인천광역시 남동구 만수6동 1037-0,37.441915,126.740316,인천광역시청,,2021-12-03,,,Y,경사로
7754,인천광역시,계양구,28245,"인천광역시 계양구 선주로56번길 5, (선주지동)",인천광역시 계양구 계양1동 108-4,37.574715,126.737592,인천광역시청,,2021-12-03,,,Y,경사로


In [23]:

obstacle_df['signgucode'] = pd.to_numeric(obstacle_df['signgucode'], errors='coerce')
obstacle_df['latitude'] = pd.to_numeric(obstacle_df['latitude'], errors='coerce')
obstacle_df['longitude'] = pd.to_numeric(obstacle_df['longitude'], errors='coerce')
print(obstacle_df.dtypes)

ctprvnnm              object
signgunm              object
signgucode             int64
rdnmadr               object
lnmadr                object
latitude             float64
longitude            float64
institutionnm         object
phonenumeric          object
referencedate         object
instt_code            object
instt_nm              object
handicapcvntlyn       object
handicapcvntltype     object
dtype: object


In [24]:
# obstacle_df DataFrame을 'C:/my_code/moduroad/data/obstacle/slope.pkl' 경로로 저장합니다.
obstacle_df.to_pickle('C:/my_code/moduroad/data/obstacle/slope.pkl')

In [52]:
file_path = 'C:/my_code/moduroad/moduroad_app/cashe_data/api_graph.pickle'

with open(file_path, 'rb') as file:
    G = pickle.load(file)

In [ ]:
import pandas as pd
import osmnx as ox
from shapely.geometry import Point
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from shapely.strtree import STRtree
import pickle
import osmnx as ox
import folium
file_path = 'C:/my_code/moduroad/moduroad_app/cashe_data/api_graph.pickle'

with open(file_path, 'rb') as file:
    G = pickle.load(file)
# Load the DataFrame from the pickle file
pickle_file_path = 'C:/my_code/moduroad/data/obstacle/slope.pkl'  # Replace with your pickle file path
obstacle_df = pd.read_pickle(pickle_file_path)

G.graph['crs'] = 'epsg:4326'

# 장애물 가까이에 있는 엣지 가중치 증가 함수 정의
def update_edge_weights(G, point, attribute_name='slope', increment=1):
    # 가장 가까운 엣지 찾기
    
    nearest_edge_nodes = ox.distance.nearest_edges(G, X=point.x, Y=point.y)  # OSMnx는 (y, x) 형식으로 좌표를 사용합니다.
    print(nearest_edge_nodes)
    # nearest_edge_nodes는 (u, v, key) 형태의 튜플입니다.
    u, v, key = nearest_edge_nodes
    print(nearest_edge_nodes)
    # 해당 엣지의 데이터 가져오기
    nearest_edge = G[u][v][key]
    print(nearest_edge)
    # 가장 가까운 엣지의 가중치 업데이트
    # 시작 지점에 마커 추가
    
    if attribute_name in nearest_edge:
    # 속성 값이 0인지 확인합니다.
        if nearest_edge[attribute_name] == 0:
            # 속성 값이 0이라면, 새로운 포인트를 포함하는 리스트로 변경합니다.
            nearest_edge[attribute_name] = [point]
        elif isinstance(nearest_edge[attribute_name], list):
            # 속성 값이 리스트인 경우, 새로운 포인트를 리스트에 추가합니다.
            nearest_edge[attribute_name].append(point)
        else:
            # 속성 값이 0이 아니고 리스트도 아닌 다른 값인 경우, 그 값을 리스트에 넣고 새로운 포인트도 추가합니다.
            current_value = nearest_edge[attribute_name]
            nearest_edge[attribute_name] = [current_value, point]
            #print("change")
        print(nearest_edge)
    
        return (u, v, key), nearest_edge
    

# 모든 장애물에 대해 가중치 업데이트 함수 실행
for index, obstacle in obstacle_df.iloc[5:].iterrows():
    latitude = obstacle['latitude']
    longitude = obstacle['longitude']

    # Point 객체 생성
    points= Point(longitude, latitude)
    # 가장 가까운 엣지의 가중치 업데이트 함수 적용
    updated_edge_info = update_edge_weights(G, points, increment=1)


In [3]:
# 저장할 파일 경로 지정
file_path_to_save = 'C:/my_code/moduroad/moduroad_app/cashe_data/api_graph_slope.pickle'

# api_graph 객체를 피클 파일로 저장
with open(file_path_to_save, 'wb') as file:
    pickle.dump(G, file)

In [ ]:
import os
import pickle
import networkx as nx

# 메인 그래프를 불러오는 함수
def load_main_graph(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

# 개별 장애물 그래프를 불러오는 함수
def load_obstacle_graph(directory, obstacle_name):
    filename = f"api_graph_{obstacle_name}.pickle"
    path = os.path.join(directory, filename)
    with open(path, 'rb') as f:
        return pickle.load(f)

def combine_graphs(main_graph, obstacle_graph, obstacle_name):
    num=0
    graph = obstacle_graph
    for u, v, data in graph.edges(data=True):
        if data.get(obstacle_name, 1) == 0:  # data[obstacle_name] 값이 0이면 해당 엣지를 처리하지 않음
            continue
        if main_graph.has_edge(u, v):
            if obstacle_name in main_graph[u][v][0]:
        
                print(main_graph[u][v][0])
                main_graph[u][v][0][obstacle_name] = data[obstacle_name]
                print(num,"change!", data[obstacle_name])
                print(main_graph[u][v][0])
                num+=1
            else:
                print("no!")
                main_graph.add_edge(u, v, **{obstacle_name: data[obstacle_name]})
        else:
            print("no!")
            main_graph.add_edge(u, v, **{obstacle_name: data[obstacle_name]})

# 메인 그래프 파일 경로
main_graph_path = 'C:/my_code/moduroad/moduroad_app/cashe_data/factor_graph.pickle'
# 장애물 그래프들이 있는 디렉토리 경로
directory_path = 'C:/my_code/moduroad/moduroad_app/cashe_data/'

# 메인 그래프 불러오기
main_graph = load_main_graph(main_graph_path)

# 수동으로 장애물 그래프 결합
# 예시: 'construction' 장애물 그래프를 불러오고 결합하는 과정
obstacle_name = 'stair_steep'  # 이 부분을 원하는 장애물 이름으로 변경하세요.
obstacle_graph = load_obstacle_graph(directory_path, obstacle_name)
combine_graphs(main_graph, obstacle_graph, obstacle_name)

# 필요한 경우 메인 그래프 저장
# with open('moduroad_app/cashe_data/combined_api_graph.pickle', 'wb') as f:
#     pickle.dump(main_graph, f)


In [ ]:
import os
import pickle
import networkx as nx

# 개별 장애물 그래프를 불러오는 함수
def load_obstacle_graph(directory, obstacle_name):
    filename = f"api_graph_{obstacle_name}.pickle"
    path = os.path.join(directory, filename)
    with open(path, 'rb') as f:
        return pickle.load(f)

def combine_graphs(main_graph, obstacle_graph, obstacle_name):
    num=0
    graph = obstacle_graph
    for u, v, data in graph.edges(data=True):
        if data.get(obstacle_name, 1) == 0:  # data[obstacle_name] 값이 0이면 해당 엣지를 처리하지 않음
            continue
        if main_graph.has_edge(u, v):
            if obstacle_name in main_graph[u][v][0]:
        
                print(main_graph[u][v][0])
                main_graph[u][v][0][obstacle_name] = data[obstacle_name]
                print(num,"change!", data[obstacle_name])
                print(main_graph[u][v][0])
                num+=1
            else:
                print("no!")
                main_graph.add_edge(u, v, **{obstacle_name: data[obstacle_name]})
        else:
            print("no!")
            main_graph.add_edge(u, v, **{obstacle_name: data[obstacle_name]})


# 장애물 그래프들이 있는 디렉토리 경로
directory_path = 'C:/my_code/moduroad/moduroad_app/cashe_data/'

# 수동으로 장애물 그래프 결합
# 예시: 'construction' 장애물 그래프를 불러오고 결합하는 과정
obstacle_name = 'slope'  # 이 부분을 원하는 장애물 이름으로 변경하세요.
obstacle_graph = load_obstacle_graph(directory_path, obstacle_name)
combine_graphs(main_graph, obstacle_graph, obstacle_name)

# 필요한 경우 메인 그래프 저장
# with open('moduroad_app/cashe_data/combined_api_graph.pickle', 'wb') as f:
#     pickle.dump(main_graph, f)


In [11]:

with open('C:/my_code/moduroad/moduroad_app/cashe_data/factor_graph.pickle', 'wb') as f:
    pickle.dump(main_graph, f)